In [32]:
#https://www.kaggle.com/anikannal/solar-power-generation-data/tasks
#https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44629

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [4]:
train = pd.read_parquet("train.parquet")
val = pd.read_parquet("val.parquet")

In [5]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']

In [6]:
encoder = OneHotEncoder(cols=cats, use_cat_names=True)
train2_ohe = encoder.fit_transform(train2[cats])
val2_ohe = encoder.transform(val2[cats])

In [7]:
import torch.nn as nn
import torch
import torch.optim as opt
import torch.nn.functional as F

In [8]:
train2_ohe.shape

(33112, 57)

# Autoencoder

In [9]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.hid1 = nn.Linear(57, 100)
        self.hid2 = nn.Linear(100, 20)
        self.hid3 = nn.Linear(20, 100)
        self.out = nn.Linear(100, 57)
    
    def forward(self, x, return_embeddings=False):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)
        
        if return_embeddings:
            return x
        
        x = F.relu(self.hid3(x))
        x = self.out(x)
        

        return x
        

In [10]:
torch.manual_seed(0)
net = AE().cuda()
criterion = nn.MSELoss()
optimizer = opt.Adam(net.parameters(), lr=1e-4)

x = torch.from_numpy(train2_ohe.values).cuda().float()
for e in range(10000):
    optimizer.zero_grad()
    
    p = net(x).cuda()
    loss = criterion(p, x)
    #print(loss.item())
    
    loss.backward()
    optimizer.step()
    
    p_numpy = p.cpu().detach().numpy()
    if e % 500 == 0:
        print(np.sqrt(mean_squared_error(train2_ohe, p_numpy)))
        

0.2744210363369018
0.19551843949297235
0.1586244387827735
0.1539853959560526
0.1472368681730391
0.13988281547015302
0.12738825269912088
0.11344051539762846
0.1023183902748767
0.0944500643454336
0.0884467591319497
0.08420787644407796
0.08051321836102
0.07717121656828248
0.07303186128507906
0.0701664512780921
0.06856314267636233
0.06758989488797663
0.0669185036289768
0.06643998355965351


# Embedding to Tree

In [11]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

x = torch.from_numpy(train2_ohe.values).cuda().float()
Xtr = net(x,return_embeddings=True).detach().cpu().numpy()
ytr = train2['Y4WIN']
mdl.fit(Xtr,ytr)

x = torch.from_numpy(val2_ohe.values).cuda().float()
Xval = net(x,return_embeddings=True).detach().cpu().numpy()

p = mdl.predict(Xval)
median_absolute_error(val2['Y4'], p)

7.099967048056477

In [12]:
Xtr

array([[ 3.4740083 ,  3.0636299 , -0.4519273 , ..., -4.5541353 ,
        -0.11379942,  1.697757  ],
       [ 2.7433536 ,  0.7541227 , -1.0567296 , ..., -4.395268  ,
        -0.64125574,  1.4667456 ],
       [ 3.2523525 ,  1.1885922 , -0.64861846, ..., -4.378553  ,
        -0.6969066 ,  2.1808126 ],
       ...,
       [ 2.3980656 ,  1.788938  ,  1.6146015 , ..., -3.0812144 ,
        -0.5506051 ,  1.5955024 ],
       [ 1.5456054 ,  1.2236401 ,  0.81726503, ..., -2.6684003 ,
        -0.19293784,  0.7746889 ],
       [ 1.9848764 ,  0.57598275,  1.0098548 , ..., -3.3337908 ,
        -0.6913406 ,  0.2916729 ]], dtype=float32)